In [ ]:
import pandas as pd
import re

In [ ]:
csv_files = [
    "sample_data/acm.csv",
    "sample_data/ieee.csv",
    "sample_data/wos.csv",
    "sample_data/scopus.csv"
]


In [ ]:
possible_title_cols = [
    "Title",
    "Document Title",
    "Article Title",
    "Paper Title",
    "TI",          # Some databases use short codes
    "doc_title"
]

In [ ]:
def normalize_title_column(df):
    # find the first matching title column
    for col in df.columns:
        if col.strip().lower() in [p.lower() for p in possible_title_cols]:
            df["title_clean"] = df[col].astype(str)
            return df

    # If no known title column found → fallback: create empty column
    df["title_clean"] = ""
    return df

In [ ]:
merged = []

for path in csv_files:
    try:
        df = pd.read_csv(path)
        df["__source_file"] = path

        # Normalize title column
        df = normalize_title_column(df)

        merged.append(df)
        print(f"Loaded: {path} | Rows: {len(df)}")
    except Exception as e:
        print(f"Could not load {path}: {e}")

merged_df = pd.concat(merged, ignore_index=True)
print(f"\nTotal merged rows: {len(merged_df)}")

Loaded: sample_data/acm.csv | Rows: 20
Loaded: sample_data/ieee.csv | Rows: 56
Loaded: sample_data/wos.csv | Rows: 40
Loaded: sample_data/scopus.csv | Rows: 22

Total merged rows: 138


In [ ]:
def clean_title(text):
    text = str(text).lower().strip()
    text = re.sub(r"[^a-z0-9 ]", " ", text)   # remove punctuation
    text = re.sub(r"\s+", " ", text)          # collapse multiple spaces
    return text

In [ ]:
merged_df["title_clean"] = merged_df["title_clean"].apply(clean_title)

In [ ]:
duplicate_mask = merged_df.duplicated(subset=["title_clean"], keep=False)
duplicates_only = merged_df[duplicate_mask].copy()
deduped_df = merged_df.drop_duplicates(subset=["title_clean"], keep="first").copy()

In [ ]:
deduped_df.to_csv("/content/deduped_output.csv", index=False)
duplicates_only.to_csv("/content/duplicates_found.csv", index=False)

In [ ]:
print("\n📊 SUMMARY")
print("——————————————")
print(f"Total merged records:     {len(merged_df)}")
print(f"Unique after dedupe:      {len(deduped_df)}")
print(f"Duplicate records found:  {len(duplicates_only)}")
print("\nFiles saved:")
print("✔ /content/deduped_output.csv")
print("✔ /content/duplicates_found.csv")


📊 SUMMARY
——————————————
Total merged records:     138
Unique after dedupe:      94
Duplicate records found:  64

Files saved:
✔ /content/deduped_output.csv
✔ /content/duplicates_found.csv


In [ ]:
# Load the deduplicated dataset from the previous step
deduped_df = pd.read_csv("/content/deduped_output.csv")

print("Columns in your data:")
print(list(deduped_df.columns))

Columns in your data:
['Item type', 'Authors', 'Title', 'Journal', 'Publication year', 'Volume', 'Issue', 'Pages', 'Publisher', 'Address', 'Proceedings title', 'Conference location', 'Date published', 'ISBN', 'ISSN', 'URLs', 'DOI', 'Abstract', 'Keywords', 'Series', '__source_file', 'title_clean', 'Document Title', 'Author Affiliations', 'Publication Title', 'Date Added To Xplore', 'Publication Year', 'Start Page', 'End Page', 'ISBNs', 'Funding Information', 'PDF Link', 'Author Keywords', 'IEEE Terms', 'Mesh_Terms', 'Article Citation Count', 'Patent Citation Count', 'Reference Count', 'License', 'Online Date', 'Issue Date', 'Meeting Date', 'Document Identifier', 'Publication Type', 'Book Authors', 'Group Authors', 'Book Group Authors', 'Researcher Ids', 'ORCIDs', 'Book Editors', 'Author - Arabic', 'Grant Principal Investigator', 'Grant Co Principal Investigator', 'Article Title', 'Article Title - SciELO', 'Article Title - SciELO.1', 'Article Title - Chinese', 'Article Title - Russian', 

In [ ]:
# Possible columns that may contain country / affiliation info
possible_country_cols = [
    "Affiliation Country",
    "Affiliations",
    "Author Affiliation",
    "Author Affiliations",
    "Affiliations 1",
    "Country",
    "Affiliation"
]

# Keep only the ones that actually exist in your dataframe
country_cols = [c for c in possible_country_cols if c in deduped_df.columns]

print("Using these columns for country detection:")
print(country_cols)

if not country_cols:
    raise ValueError("No country/affiliation columns found. Please update 'possible_country_cols' list.")

Using these columns for country detection:
['Author Affiliations']


In [ ]:
target_countries = [
    "Pakistan",
    "Bangladesh",
    "Nepal",
    "Sri Lanka",
    "Bhutan",
    "Maldives",
    "India",
    "Afghanistan",
    "South Asia",   # in case region appears
    "South Asian"
]

In [ ]:
def find_countries_in_row(row):
    text = ""
    for col in country_cols:
        text += " " + str(row[col])
    text_lower = text.lower()

    found = []
    for country in target_countries:
        if country.lower() in text_lower:
            found.append(country)
    # return unique list
    return list(set(found))

# Apply detection
deduped_df["countries_detected"] = deduped_df.apply(find_countries_in_row, axis=1)

# Explode list of countries into long format
exploded = deduped_df.explode("countries_detected")

# Remove rows with no detected country
exploded = exploded[exploded["countries_detected"].notna() & (exploded["countries_detected"] != "")]

# Count papers per country (by unique title_clean if available, else by index)
paper_id_col = "title_clean" if "title_clean" in deduped_df.columns else None

if paper_id_col:
    country_counts = exploded.groupby("countries_detected")[paper_id_col].nunique().reset_index()
else:
    country_counts = exploded["countries_detected"].value_counts().reset_index()
    country_counts.columns = ["countries_detected", "count"]

country_counts = country_counts.sort_values(by=country_counts.columns[-1], ascending=False)

print("\n📊 Papers per country:")
print(country_counts)

# Save to CSV
country_counts.to_csv("/content/papers_per_country.csv", index=False)
print("\nSaved summary to /content/papers_per_country.csv")


📊 Papers per country:
  countries_detected  title_clean
0         Bangladesh           27
1           Pakistan           15
2          Sri Lanka            6

Saved summary to /content/papers_per_country.csv
